<a href="https://colab.research.google.com/github/habibsifat/User-Friendly-Bangla-Writer-A-Deep-Learning-Based-Approach/blob/master/Predict_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Import**

In [0]:
from __future__ import print_function
__author__ = 'maxim'
import numpy as np
import pandas as pd
import gensim
import string
from keras.layers.core import Dense, Activation, Dropout
from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils.data_utils import get_file

Using TensorFlow backend.


# Load

In [0]:
!ls 'drive/My Drive/Colab Notebooks/Capstone/Final/final_corpus.txt'


'drive/My Drive/Colab Notebooks/Capstone/Final/final_corpus.txt'


In [0]:
corpus_file = 'drive/My Drive/Colab Notebooks/Capstone/Final/final_corpus.txt'

In [0]:
print('\nPreparing the  sentences...')
sentences = []
with open(corpus_file, 'r', encoding="utf-8-sig") as f:
    for line in f:
        sentences.append(line.split())
        
print('Num sentences:', len(sentences))


Preparing the  sentences...
Num sentences: 6476964


# W2V

In [0]:
!pip install gensim
from gensim.models import Word2Vec

In [0]:
print('\nTraining word2vec...')
word_model = Word2Vec(sentences, size=300, min_count=1, window=5, iter=100)
pretrained_weights = word_model.wv.syn0 
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)

#Save partly trained model
#word_model.save('w2v_model.h5')


Training word2vec...
Result embedding shape: (8789, 300)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
#Save partly trained model
model_path = 'drive/My Drive/Colab Notebooks/Capstone/Final/100_w2v_model.h5'
word_model.save(model_path)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def word2idx(word):
  return word_model.wv.vocab[word].index
def idx2word(idx):
  return word_model.wv.index2word[idx]

#Train

In [0]:
!ls 'drive/My Drive/Colab Notebooks/Capstone/Clean_txt/All Lenth/lenth3.txt'

'drive/My Drive/Colab Notebooks/Capstone/Clean_txt/All Lenth/lenth3.txt'


In [0]:
corpus_file = 'drive/My Drive/Colab Notebooks/Capstone/Clean_txt/All Lenth/lenth3.txt'
print('\nPreparing the sentences...')
sentences = []
with open(corpus_file, 'r', encoding="utf-8-sig") as f:
    for line in f:
        sentences.append(line.split())
print('Num sentences:', len(sentences))


Preparing the sentences...
Num sentences: 4638


In [0]:
max_sentence_len = 3
print('\nPreparing the data for LSTM...')
x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)

In [0]:
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.2,random_state=10)

In [0]:
print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=512,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=512, return_sequences=False))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y,batch_size=128,epochs=5,shuffle=True, validation_data=(test_x, test_y))


Training LSTM...
Train on 3710 samples, validate on 928 samples
Epoch 1/5
3710/3710 [==============================] - 4s 1ms/step - loss: 1.9530 - acc: 0.9655 - val_loss: 4.2551e-05 - val_acc: 1.0000
Epoch 2/5
3710/3710 [==============================] - 1s 248us/step - loss: 4.2585e-05 - acc: 1.0000 - val_loss: 4.2576e-05 - val_acc: 1.0000
Epoch 3/5
3710/3710 [==============================] - 1s 245us/step - loss: 4.2580e-05 - acc: 1.0000 - val_loss: 4.2571e-05 - val_acc: 1.0000
Epoch 4/5
3710/3710 [==============================] - 1s 250us/step - loss: 4.2588e-05 - acc: 1.0000 - val_loss: 4.2573e-05 - val_acc: 1.0000
Epoch 5/5
3710/3710 [==============================] - 1s 247us/step - loss: 4.2583e-05 - acc: 1.0000 - val_loss: 4.2583e-05 - val_acc: 1.0000


In [0]:
#Load  model
from keras.models import load_model
model = load_model('model.h5')

In [0]:
max_sentence_len = 10
corpus_file='drive/My Drive/Colab Notebooks/Capstone/Clean_txt/All Lenth/lenth10.txt'
print('\nPreparing the sentences...')
sentences = []
with open(corpus_file, 'r', encoding="utf-8-sig") as f:
    for line in f:
        sentences.append(line.split())
print('Num sentences:', len(sentences))

print('\nPreparing the data for LSTM...')
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)
print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=emdedding_size,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=emdedding_size, return_sequences=False))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(train_x, train_y,
          batch_size=128,
          epochs=1,
callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])


In [0]:
#Save partly trained model
model.save('model.h5')

# Predict

In [0]:
def generate_next(text, num_generated=1):
  word_idxs = [word2idx(word) for word in text.split()]
  #print(word_idxs)
  #Str1 = input("Enter word :")
  Str1 = "অনেক"
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs)) #print dimension
    max_word = prediction[-1].argsort()[-1:][::-1]
    #print(max_word)
    j=1
    for idx in max_word:
      word_idxs.append(idx)
      Distance = levenshtein_ratio_and_distance(Str1,idx2word(idx))
      Ratio = levenshtein_ratio_and_distance(Str1,idx2word(idx),ratio_calc = True)
      #print("Max word %d : %s " % (j,idx2word(idx)))
      #print("Distance : %s And Ratio : %s " % (Distance,Ratio))
      j+=1
    #print(word_idxs)
  return ' '.join(idx2word(idx) for idx in word_idxs)

In [0]:
def on_epoch_end(epoch, _):
  #print('\n For Lenth-5 Generating text after epoch: %d' % epoch)
  texts = [
    '* তড়িৎ প্রকৌশল UNK ',
      '* এটি UNK বাংলা '
  ]
  for text in texts:
    sample = generate_next(text)
    #print('%s -> %s' % (text, sample))

model.fit(train_x, train_y,
          batch_size=128,
          epochs=10,
callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

model.summary()

Epoch 1/10
4638/4638 [==============================] - 1s 121us/step - loss: 2.1538
Epoch 2/10
4638/4638 [==============================] - 1s 123us/step - loss: 2.1549
Epoch 3/10
4638/4638 [==============================] - 1s 122us/step - loss: 2.1610
Epoch 4/10
4638/4638 [==============================] - 1s 126us/step - loss: 2.1565
Epoch 5/10
4638/4638 [==============================] - 1s 123us/step - loss: 2.1565
Epoch 6/10
4638/4638 [==============================] - 1s 121us/step - loss: 2.1563
Epoch 7/10
4638/4638 [==============================] - 1s 125us/step - loss: 2.1565
Epoch 8/10
4638/4638 [==============================] - 1s 121us/step - loss: 2.1523
Epoch 9/10
4638/4638 [==============================] - 1s 122us/step - loss: 2.1561
Epoch 10/10
4638/4638 [==============================] - 1s 120us/step - loss: 2.1544
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param 

# Edit_Distance

In [0]:
import numpy as np
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return " {} ".format(distance[row][col])

In [0]:
Str1 = "Apple Inc."
Str2 = "Apple Inc"
Distance = levenshtein_ratio_and_distance(Str1,Str2)
print(Distance)
Ratio = levenshtein_ratio_and_distance(Str1,Str2,ratio_calc = True)
print(Ratio)

In [0]:
!pip install Levenshtein

In [0]:
import Levenshtein as lev
Str1 = "Apple Inc."
Str2 = "apple Inc"
Distance = lev.distance(Str1.lower(),Str2.lower()),
print(Distance)
Ratio = lev.ratio(Str1.lower(),Str2.lower())
print(Ratio)

# Give Suggesion

In [0]:
#Load  word_model
from keras.models import load_model
word_model = load_model('word_model.h5')
#Load  model
from keras.models import load_model
model = load_model('model.h5')

In [0]:
i=1
for idx in max_word:
      word_idxs.append(idx)
      print("Max word %s : %s " % (i,idx2word(idx)))
      i+=1

Fuzzy

In [0]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

In [0]:
Str1 = "Los Angeles Lakers"
Str2 = "Lakers"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
print(Ratio)
print(Partial_Ratio)

# Paragraph Correction

In [0]:
def generate_next_para(text, str1,num_generated=1):
  word_idxs = [word2idx(word) for word in text.split()]
  #print(word_idxs)
  #Str1 = input("Enter word :")
  #Str1 = "অনেক"
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs)) #print dimension
    max_word = prediction[-1].argsort()[-10:][::-1]
    #print(max_word)
    j=1
    for idx in max_word:
      word_idxs.append(idx)
      Distance = levenshtein_ratio_and_distance(Str1,idx2word(idx))
      Ratio = levenshtein_ratio_and_distance(Str1,idx2word(idx),ratio_calc = True)
      print("Max word %d : %s " % (j,idx2word(idx)))
      print("Distance : %s And Ratio : %s " % (Distance,Ratio))
      j+=1
    #print(word_idxs)
  return ' '.join(idx2word(idx) for idx in word_idxs)

In [0]:
#para = "* শারাদিন  আজ বৃতি । রাতে  ঝর হাওয়া বএ যেতে পারে ।"
para = "* শারাদিন"
# using split() 
# to extract words from string 
res = para.split()  
# printing result 
print ("The list of words is : " +  str(res)) 

In [0]:
for i in range(0,len(res)-1):
  print(i)
  word = res[i]
  print(word) 
  word2 = res[i+1]
  print(word2) 
  sample = generate_next_para(word,word2)
  print('%s -> %s' % (text, sample))